In [ ]:
!pip install huggingface_hub > /dev/null

In [1]:
from langchain.chat_models import ChatOpenAI
from kor.extraction import create_extraction_chain
from kor.nodes import Object, Text
from kor import create_extraction_chain
from langchain.llms import OpenAI

In [137]:
import os

os.environ["HUGGINGFACEHUB_API_TOKEN"] = "mettete le vostre api stronzi scrocconi"
os.environ["OPENAI_API_KEY"] = "la mamma di morelli ha i baffi"

In [138]:
##llm = OpenAI(temperature=0.9)
from langchain import HuggingFaceHub

repo_id = "google/flan-t5-small" # See https://huggingface.co/models?pipeline_tag=text-generation&sort=downloads for some other options
#repo_id = "tiiuae/falcon-7b"
llm = HuggingFaceHub(repo_id=repo_id, model_kwargs={"temperature":0.2, "max_length":64})

llm = OpenAI(temperature=0.2)

/Users/prollo/miniforge3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [139]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

examples = [
  {
    "query": "I want to know the wstETH balance of 0xFe8e15ae884524eFfc2fe91dF6f5BA40D8533A92 on Gnosis",
    "answer": "action:balance,token:wstETH,address:0xFe8e15ae884524eFfc2fe91dF6f5BA40D8533A92,chain:Gnosis,amount:100"
  },
  {
    "query":"I want to swap 100 XDAI for DAI on Gnosis",
    "answer":"action:swap,token1:XDAI,token2:DAI,chain:Gnosis,amount:100"
  },
  {
    "query": "I want to send or transfer 100 WETH to 0xFe8e15ae884524eFfc2fe91dF6f5BA40D8533A92 on Gnosis",
    "answer":"action:transfer,token:WETH,chain:Gnosis,address:0xFe8e15ae884524eFfc2fe91dF6f5BA40D8533A92,amount:100"
  },
  {
    "query": "I want to swap 100 DAI to USDC on Chiado",
    "answer":"action:swap,token1:DAI,token2:USDC,chain:Chiado,amount:100"
  },
  
]


example_template = """
User: {query}
AI: {answer}
"""


# create a prompt example from above template
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix = """The following are exerpts from conversations with an AI
assistant. The assistant is a web3 developer experts that extract parameters from text. Here are some
examples: 
"""

# and the suffix our user input and output indicator
suffix = """
User: {query}
AI: """

# now create the few shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

query = "I want to swap 150 BTC to USDC on Gnosis"
print(few_shot_prompt_template.format(query=query))

The following are exerpts from conversations with an AI
assistant. The assistant is a web3 developer experts that extract parameters from text. Here are some
examples: 



User: I want to know the wstETH balance of 0xFe8e15ae884524eFfc2fe91dF6f5BA40D8533A92 on Gnosis
AI: action:balance,token:wstETH,address:0xFe8e15ae884524eFfc2fe91dF6f5BA40D8533A92,chain:Gnosis,amount:100



User: I want to swap 100 XDAI for DAI on Gnosis
AI: action:swap,token1:XDAI,token2:DAI,chain:Gnosis,amount:100



User: I want to send or transfer 100 WETH to 0xFe8e15ae884524eFfc2fe91dF6f5BA40D8533A92 on Gnosis
AI: action:transfer,token:WETH,chain:Gnosis,address:0xFe8e15ae884524eFfc2fe91dF6f5BA40D8533A92,amount:100



User: I want to swap 100 DAI to USDC on Chiado
AI: action:swap,token1:DAI,token2:USDC,chain:Chiado,amount:100



User: I want to swap 150 BTC to USDC on Gnosis
AI: 


In [140]:
llm(few_shot_prompt_template.format(query=query))

' action:swap,token1:BTC,token2:USDC,chain:Gnosis,amount:150'